# 1) Imports

In [28]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import great_circle
import warnings
import unidecode
import re

warnings.filterwarnings('ignore')

# 2) Coleta dos Dados

In [29]:
imputer = IterativeImputer(random_state=42)
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")

## 2.1) Lista de municípios do Sertão

In [30]:
mun_ser = pd.read_excel(r'../Dados/Municipios/lista-1262municipios-semiarido-2017.xlsx', header = 2).iloc[1:]
mun_ser = mun_ser[['Cod IBGE','UF', 'Município']]
mun_ser.columns = ['IBGE7', 'UF', 'NOME']

mun_cord = pd.read_csv(r'../Dados/Municipios/municipios.csv')
mun_cord = mun_cord[['codigo_ibge', 'latitude', 'longitude']]
mun_cord.columns = ['IBGE7', 'LATITUDE', 'LONGITUDE']

mun_ser = mun_ser.merge(mun_cord, on ='IBGE7', how = 'left')
mun_ser.to_csv('../Dados/Views/municipios_sertao.csv')
mun_ser

,IBGE7,UF,NOME,LATITUDE,LONGITUDE
0,2700300,AL,ARAPIRACA,-9.75487,-36.6615
1,2700706,AL,BATALHA,-9.67420,-37.1330
2,2700904,AL,BELO MONTE,-9.82272,-37.2770
3,2701209,AL,CACIMBINHAS,-9.40121,-36.9911
4,2701605,AL,CANAPI,-9.11932,-37.5967
...,...,...,...,...,...
1256,2806008,SE,RIBEIRÓPOLIS,-10.53570,-37.4380
1257,2807006,SE,SÃO MIGUEL DO ALEIXO,-10.38470,-37.3836
1258,2807105,SE,SIMÃO DIAS,-10.73870,-37.8097
1259,2807303,SE,TELHA,-10.20640,-36.8818


## 2.2) Dados de registros administrativos 

In [31]:
def calcular_populacao_total(municipio, municipios_df):
    populacao_total = municipio['POP_TOT']  
    for _, row in municipios_df.iterrows():
        distancia = great_circle((municipio['LATITUDE'], municipio['LONGITUDE']), (row['LATITUDE'], row['LONGITUDE'])).kilometers
        if distancia <= 30:
            populacao_total += row['POP_TOT']
    return populacao_total

metrics = [
    'ANO', 
    'IBGE7',
    'DIST_EF_PUB',
    'DIST_EM_PUB',
    'IDEB_AI',
    'IDEB_AF',
    'DOCSUP_EF_PUB',
    'DOCSUP_EM_PUB',
    'TXNASC7C','TXNBAIXOP',
    'PINTERSAP',
    'PINTERDRSAI',
    'REN_PIBPC_D',
    'PDEFAGUA',
    'PDEFESGOTO',
    'PANALF15',
    'PIND_POS',
    'SNIS_PAGUA',
    'POP_TOT', 
    "PMATPUB_EF", 
    "PMATPUB_EM"
]

reg_adm_mun = pd.read_excel(r'../Dados/Registros Administrativos/dados_registros_administrativos_total_2012_2017.xlsx', sheet_name = 'MUNICÍPIO')
reg_adm_mun = reg_adm_mun[reg_adm_mun['IBGE7'].isin(mun_ser['IBGE7'])]
reg_adm_mun = reg_adm_mun[metrics]

num_metrics = [
    'ANO', 
    'DIST_EF_PUB', 
    'DIST_EM_PUB', 
    'IDEB_AI',
    'IDEB_AF', 
    'DOCSUP_EF_PUB', 
    'DOCSUP_EM_PUB', 
    'TXNASC7C', 
    'TXNBAIXOP',
    'PINTERSAP', 
    'PINTERDRSAI', 
    'REN_PIBPC_D', 
    'PDEFAGUA', 
    'PDEFESGOTO',
    'PANALF15', 
    'PIND_POS', 
    'SNIS_PAGUA', 
    'POP_TOT', 
    "PMATPUB_EF", 
    "PMATPUB_EM"
]

reg_adm_mun[num_metrics] = imputer.fit_transform(reg_adm_mun[num_metrics])
reg_adm_mun = reg_adm_mun.drop(columns='ANO')
reg_adm_mun = reg_adm_mun.groupby(['IBGE7']).mean().reset_index()

cidades = mun_ser.copy()

cidades = cidades.merge(reg_adm_mun, on = 'IBGE7', how = 'inner')

cidades['POP_TOT_30KM'] = 0

for index, municipio in cidades.iterrows():
    populacao_total_30km = calcular_populacao_total(municipio, cidades)
    cidades.at[index, 'POP_TOT_30KM'] = populacao_total_30km

cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades.to_csv('../Dados/Views/registros_administrativos.csv')
cidades

,IBGE7,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,TXNASC7C,TXNBAIXOP,PINTERSAP,...,REN_PIBPC_D,PDEFAGUA,PDEFESGOTO,PANALF15,PIND_POS,SNIS_PAGUA,POP_TOT,PMATPUB_EF,PMATPUB_EM,POP_TOT_30KM
0,2700300,28.190028,42.715813,4.682072,3.682635,54.40,56.96,60.158262,10.090574,18.636148,...,11.572834,25.485645,73.903400,16.467925,40.823366,97.486,230975.6,74.732,77.548,582087.0
1,2700706,30.940427,36.601658,4.488950,3.591572,12.62,41.94,55.029615,7.258357,30.860307,...,5.504959,45.221938,83.366145,18.163104,66.975186,88.194,18487.6,89.920,97.592,120483.6
2,2700904,34.584857,53.005007,4.091777,3.176668,27.12,31.84,53.794486,8.063568,20.308137,...,6.951375,59.669250,82.802119,23.850627,43.986591,91.050,6774.4,100.000,100.000,114803.4
3,2701209,39.071078,42.736296,4.085816,3.310474,49.04,45.22,52.443551,6.698345,17.231791,...,5.446129,76.937884,62.727547,22.902594,52.875393,32.604,10815.6,91.742,99.436,141797.4
4,2701605,31.912620,36.839799,4.673732,3.635387,20.84,32.34,52.780364,8.178384,12.902533,...,3.952766,73.647852,57.191788,24.767916,39.558598,94.054,17958.4,100.000,100.000,148162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2806008,33.124036,39.388836,4.178352,3.548260,64.68,77.28,73.853258,6.865284,8.830588,...,9.191654,27.113079,37.090343,10.703906,48.340042,96.058,18359.0,80.506,100.000,100748.8
1257,2807006,31.585497,47.837042,3.932061,3.191697,62.08,66.38,55.382784,8.533114,16.568992,...,6.680577,35.514471,48.649799,14.764503,51.751195,95.810,3901.6,100.000,100.000,127390.8
1258,2807105,35.961743,43.788264,4.292358,3.462631,61.38,82.96,54.129258,7.443652,20.306567,...,8.784990,46.308771,41.600849,15.596398,61.084310,97.866,40522.2,85.434,89.606,120552.4
1259,2807303,29.181166,46.235068,4.399357,3.537064,45.66,71.74,59.483911,7.337448,12.372443,...,6.711751,19.235246,26.120017,13.385993,62.912364,98.560,3169.6,100.000,100.000,81132.8


## 2.3) Dados meteorológicos 

In [32]:
def calcular_distancia(cidade, estacao):
    cidade_coord = (cidade['LATITUDE'], cidade['LONGITUDE'])
    estacao_coord = (estacao['LATITUDE'], estacao['LONGITUDE'])
    return great_circle(cidade_coord, estacao_coord).kilometers

ufs = list(set(mun_ser['UF']))
est_met = pd.DataFrame()

num_columns = [
    'LATITUDE', 
    'LONGITUDE',
    'ALTITUDE',
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
    "RADIACAO GLOBAL (Kj/m²)",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    "VENTO, VELOCIDADE HORARIA (m/s)"
]

for year in [2020, 2021, 2022]:
    
    file_path = f'../Dados/INMET/INMET_{year}.csv'
    file = pd.read_csv(file_path)
    file = file[
        [
            'Data', 
            'UF', 
            'ESTACAO', 
            'LATITUDE', 
            'LONGITUDE',
            'ALTITUDE',
            "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
            "RADIACAO GLOBAL (Kj/m²)",
            "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
            "VENTO, VELOCIDADE HORARIA (m/s)"
        ]
    ]
    file = file[file['UF'].isin(ufs)]
    file = file.dropna(axis = 0, thresh = 9)
    file[num_columns] = imputer.fit_transform(file[num_columns])
    file = file.groupby(['Data', 'UF', 'ESTACAO','LATITUDE', 'LONGITUDE','ALTITUDE']).agg(
        {
            "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": 'sum', 
            "RADIACAO GLOBAL (Kj/m²)":'mean',
            "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": 'mean',
            "VENTO, VELOCIDADE HORARIA (m/s)":'mean'
            
        }
    ).reset_index()
    est_met = pd.concat([est_met, file], ignore_index=True)

est_met = est_met.drop(columns = 'Data')
est_met = est_met.groupby(['ESTACAO', 'UF']).mean().reset_index()
est_met.columns = [
    'ESTACAO', 
    'UF', 
    'LATITUDE', 
    'LONGITUDE', 
    'ALTIUDE',
    'PREC_MED', 
    'RED_MED',
    'TEMP_MED', 
    'VEL_MED'
]

cidades = mun_ser.copy()

for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in est_met.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    estacao_mais_proxima = est_met.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'PREC_MED'] = estacao_mais_proxima['PREC_MED']
    cidades.at[index_cidade, 'RED_MED'] = estacao_mais_proxima['RED_MED']
    cidades.at[index_cidade, 'TEMP_MED'] = estacao_mais_proxima['TEMP_MED']
    cidades.at[index_cidade, 'VEL_MED'] = estacao_mais_proxima['VEL_MED']
    
cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades.to_csv('../Dados/Views/dados_meteorologicos.csv')
cidades

,IBGE7,PREC_MED,RED_MED,TEMP_MED,VEL_MED
0,2700300,2.708739,1051.169614,26.157050,1.553386
1,2700706,1.953488,1318.900553,27.754333,1.427155
2,2700904,1.953488,1318.900553,27.754333,1.427155
3,2701209,0.573371,1585.450518,26.482644,1.062182
4,2701605,1.511106,1437.777193,27.047011,2.518969
...,...,...,...,...,...
1256,2806008,0.993684,1146.244737,27.008460,3.015386
1257,2807006,1.063896,1739.639845,27.404957,2.741722
1258,2807105,0.444873,1370.066485,26.034025,0.710818
1259,2807303,4.157919,1621.663140,26.874591,1.169412


## 2.4) Dados de recursos hídricos

In [33]:
atl_irr = pd.read_excel(r'../Dados/ANA/Atlas Irrigacao - Area Atual e Potencial.xlsx',header = 6).iloc[6:]
atl_irr = atl_irr[['Código','Área Total Irrigada', 'AAI - Potencial Total', 'AAI - Potencial Efetivo']]
atl_irr = atl_irr[atl_irr['Código'].isin(mun_ser['IBGE7'])]
atl_irr.columns = ['IBGE7','AREA_IRRIGADA_TOT', 'AREA_IRRIGADA_POT', 'AREA_IRRIGADA_POT_E']
atl_irr.to_csv('../Dados/Views/recursos_hidricos.csv')
atl_irr

,IBGE7,AREA_IRRIGADA_TOT,AREA_IRRIGADA_POT,AREA_IRRIGADA_POT_E
465,2100907,56.000000,829.401141,3.140248
653,2112209,666.262426,4269.334204,0.000000
667,2200053,96.000000,0.000000,0.000000
670,2200251,0.000000,1240.907963,0.000000
671,2200277,2.000000,0.000000,0.000000
...,...,...,...,...
3076,3170651,353.204987,1106.772079,142.399238
3079,3170800,4133.645939,37599.860831,1881.799953
3080,3170909,197.000000,400.497303,1.268556
3082,3171030,4822.626564,295.487156,3.395124


## 2.3) Dados do censo

In [34]:
censo = pd.read_excel(r'../Dados/Censo/Censo_municipal_estadual_nacional.xlsx', sheet_name = 'MUN 91-00-10')
censo = censo[(censo['Codmun7'].isin(mun_ser['IBGE7']))&(censo['ANO']==2010)]
censo_colunas = ['Codmun7', 'IDHM', 'T_BANAGUA','T_DENS','T_LIXO','T_LUZ','AGUA_ESGOTO',
                 'PAREDE', 'PIND', 'PINDCRI', 'T_DES18M', 'I_FREQ_PROP', 
                 'T_FBPRE', 'T_FBSUPER', 'T_ATRASO_1_BASICO', 'T_ATRASO_1_FUND', 'T_ATRASO_1_MED',
                 'T_ATRASO_2_BASICO', 'T_ATRASO_2_FUND', 'T_ATRASO_2_MED', 'T_ANALF11A14',
                 'T_ANALF15A17', 'T_ANALF15M', 'T_ANALF18A24', 'T_ANALF18M', 'T_ANALF25A29',
                 'T_ANALF25M' ]

censo = censo[censo_colunas]
censo.columns = censo_colunas
censo.to_csv('../Dados/Views/censo.csv')
censo

,Codmun7,IDHM,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,PIND,PINDCRI,...,T_ATRASO_2_BASICO,T_ATRASO_2_FUND,T_ATRASO_2_MED,T_ANALF11A14,T_ANALF15A17,T_ANALF15M,T_ANALF18A24,T_ANALF18M,T_ANALF25A29,T_ANALF25M
11594,2100907,0.521,31.94,42.14,71.79,95.54,23.23,33.06,40.55,49.69,...,32.33,25.21,12.84,11.16,9.96,33.70,13.71,36.20,22.16,42.60
11782,2112209,0.649,76.38,39.60,77.55,98.66,7.39,17.06,9.62,15.38,...,24.72,19.87,10.84,6.32,3.28,17.26,4.16,18.49,7.86,22.40
11796,2200053,0.528,19.28,36.52,99.22,98.99,23.32,0.53,32.12,46.41,...,28.15,20.35,15.65,7.49,4.62,30.41,6.79,32.67,14.49,39.08
11799,2200251,0.531,48.55,24.04,97.24,86.10,10.77,0.21,28.91,41.95,...,38.92,35.54,6.81,15.70,11.72,44.40,14.57,46.90,27.83,53.81
11800,2200277,0.585,72.12,34.60,97.43,98.34,3.89,0.00,22.97,32.09,...,30.75,26.90,3.28,4.73,5.70,35.26,6.92,37.95,17.77,45.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14205,3170651,0.634,75.25,18.35,88.58,96.62,16.25,0.16,15.35,21.42,...,13.82,9.77,3.17,2.81,1.82,25.27,3.03,27.77,8.74,34.48
14208,3170800,0.666,85.76,26.52,97.24,99.13,9.12,0.63,4.81,6.75,...,20.40,17.76,9.39,1.85,1.15,12.00,1.94,12.97,3.02,15.69
14209,3170909,0.594,66.33,37.32,67.62,98.34,17.92,0.38,22.81,30.83,...,25.06,20.02,18.77,2.58,2.42,24.45,4.64,27.06,8.40,33.46
14211,3171030,0.584,59.85,40.46,95.64,95.35,16.28,4.68,14.73,18.72,...,26.21,22.74,9.13,6.76,4.15,26.85,6.22,30.03,12.69,37.53


## 2.4) Qualidade da água

In [35]:
ql_agua = pd.read_csv(r'../Dados/ANA/Indicadores de Qualidade da Agua.csv')
ql_agua = ql_agua[['CDESTACAO','CORPODAGUA', 'SGUF','LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']]
ql_agua = ql_agua[ql_agua['SGUF'].isin(ufs)]
ql_agua = ql_agua.dropna(axis = 0, thresh = 3)

num_columns = ['LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']

ql_agua[num_columns] = imputer.fit_transform(ql_agua[num_columns])
ql_agua['QUAL_MED_AGUA'] = (ql_agua['MED_2021'] + ql_agua['MED_2020'])/2
ql_agua = ql_agua.reset_index().drop(columns='index')
ql_agua

cidades = mun_ser.copy()
dist_corpo_agua_mais_prox = []

for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in ql_agua.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    dist_corpo_agua_mais_prox.append(min(distancias))
    estacao_mais_proxima = ql_agua.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'QUAL_MED_AGUA'] = estacao_mais_proxima['QUAL_MED_AGUA']

cidades['DIST_CORPO_AGUA'] = dist_corpo_agua_mais_prox
cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades.to_csv('../Dados/Views/qualidade_da_agua.csv')
cidades

,IBGE7,QUAL_MED_AGUA,DIST_CORPO_AGUA
0,2700300,62.439167,42.035186
1,2700706,65.300843,24.642419
2,2700904,65.300843,3.297594
3,2701209,65.300843,58.541833
4,2701605,66.893107,68.942666
...,...,...,...
1256,2806008,64.911294,1.388835
1257,2807006,64.911294,16.508752
1258,2807105,65.088090,2.685846
1259,2807303,64.553680,4.298665


## 2.5) Dados de transporte

In [36]:
trans = pd.read_excel('../Dados/Transportes/transport_cost.xlsx')
trans = trans[['codigo_ibge','transportation_cost']].groupby('codigo_ibge').min().reset_index()
trans.columns = ['IBGE7', 'TRANSPORT_COST']
trans.to_csv('../Dados/Views/custo_de_transporte.csv')
trans

,IBGE7,TRANSPORT_COST
0,1400100,40478.947612
1,1400282,40326.183579
2,1501402,33091.358691
3,1501600,32666.086719
4,1504802,35154.250138
...,...,...
1380,5213806,25385.396775
1381,5214051,27720.752548
1382,5217203,27268.044018
1383,5219803,26161.653604


## 2.6) Variáveis educacionais

In [37]:
edu_raw_2020 = pd.read_csv(r"..\Dados\Q EDU\Dados_QEdu_Analitico_2020.csv", delimiter=";")
edu_2020 = edu_raw_2020.copy()


codigos_ne = list(range(21, 30))
edu_2020_nordeste = edu_2020[edu_2020["COD UF"].isin(codigos_ne)]

edu_2020_nordeste["Total de escolas estaduais"] = edu_2020_nordeste["Total de escolas estaduais"].str.replace(",", ".")
edu_2020_nordeste["Total de escolas municipais"] = edu_2020_nordeste["Total de escolas municipais"].str.replace(",", ".")

edu_2020_nordeste[[ 
                "Total de escolas estaduais", 
                "Total de escolas municipais"]] = edu_2020_nordeste[[ 
                                                                    "Total de escolas estaduais", 
                                                                    "Total de escolas municipais"]].astype('float').astype('Int64')

imputer_edu = SimpleImputer(strategy="mean")
dados_imputados = imputer_edu.fit_transform(edu_2020_nordeste[[ 
                                                           "Total de escolas estaduais", 
                                                           "Total de escolas municipais"]])

edu_2020_nordeste[["Total de escolas estaduais", "Total de escolas municipais"]] = dados_imputados.round()



# Calculando o número de escolas municipais em um Município:
edu_2020_nordeste["Total_escolas_publicas_mun_est"] = edu_2020_nordeste['Total de escolas estaduais'] \
    + edu_2020_nordeste['Total de escolas municipais']

df_edu_2020 = edu_2020_nordeste[["COD Municipio", "Total_escolas_publicas_mun_est"]]
df_edu_2020.set_index("COD Municipio", inplace=True)


# Calculando o número de matrículas por população:
total_matriculas =  edu_2020_nordeste[["COD Municipio", "Total de matrículas"]]
total_matriculas.set_index("COD Municipio", inplace=True)
df_edu_2020 = df_edu_2020.merge(total_matriculas, how="inner", left_index=True, right_index=True)


# Importando dados do Censo 2020 para o feature Engineering:
censo_raw_2020 = pd.read_excel(r"..\Dados\Censo\estimativa_dou_2020.xls", header=1, sheet_name="Municípios", dtype={"COD. MUNIC": str})
censo_2020 = censo_raw_2020.copy()
censo_2020_ne = censo_2020[censo_2020["COD. UF"].isin(codigos_ne)]
censo_2020_ne["COD. UF"] = censo_2020_ne["COD. UF"].astype(int).astype(str)

censo_2020_ne["CODIGO_MUNIC"] = censo_2020_ne["COD. UF"] +  censo_2020_ne["COD. MUNIC"]
censo_2020_ne.drop(columns=["COD. UF", "COD. MUNIC", "NOME DO MUNICÍPIO", "UF"], inplace=True)
censo_2020_ne.set_index("CODIGO_MUNIC", inplace=True)

resultado = censo_2020_ne["POPULAÇÃO ESTIMADA"].str.split("(").to_frame()
index = resultado.dropna().index
valores = [i[0] for i in resultado.dropna()["POPULAÇÃO ESTIMADA"]]

valores_serie = pd.Series(valores, index=index).str.strip()
censo_2020_ne["POPULAÇÃO ESTIMADA"].loc[index] = valores_serie.values
censo_2020_ne["POPULAÇÃO ESTIMADA"] = censo_2020_ne["POPULAÇÃO ESTIMADA"].astype(int)

# Juntando o dataset de educação com a população:
df_edu_2020 = df_edu_2020.merge(censo_2020_ne, how="inner", left_index=True, right_index=True)
df_edu_2020.index.name = "COD_MUNIC"

# Feature engineering:
df_edu_2020["TOT_ESC_POR_POP"] = df_edu_2020["Total_escolas_publicas_mun_est"]/ df_edu_2020["POPULAÇÃO ESTIMADA"]
df_edu_2020["TOT_MAT_POR_POP"] = df_edu_2020["Total de matrículas"]/ df_edu_2020["POPULAÇÃO ESTIMADA"]


df_edu_2020.drop(columns=["POPULAÇÃO ESTIMADA","Total_escolas_publicas_mun_est",
                          "Total de matrículas"], inplace=True)

df_edu_2020.head()

# Salvando a View de educação:
df_edu_2020.to_csv(r"..\Dados\Views\dados_edu_2020.csv")

## 2.7) Dados de produção agrícola municipal

In [38]:
def limpar_nome(nome):
    nome_sem_acentos = unidecode.unidecode(nome)  # Remove acentos
    nome_sem_codigo_estado = re.sub(r'\(\w+\)', '', nome_sem_acentos)  # Remove a sequência "(XX)"
    return nome_sem_codigo_estado.upper()  # Converte para maiúsculas

def limpar_dataframe(df, valor):
    
    municipios = df['Brasil e Município'].apply(limpar_nome)
    new_df = pd.DataFrame()
    for i in range(0,3):
        new_year_df = df.iloc[:, 1 + 37 * i : 37 * (i + 1)]
        new_year_df.columns = [column.rstrip('.1').rstrip('.2') for column in new_year_df.columns]
        new_year_df.insert(0, 'MUNICIPIO', list(municipios))
        new_df = pd.concat([new_df, new_year_df], ignore_index = True)
    new_df = new_df.replace(['-', '...'], 0)
    new_df = new_df.fillna(0)
    new_df = new_df.melt(id_vars=['MUNICIPIO'], var_name='PRODUTO', value_name=valor)
    new_df[valor] = new_df[valor].astype('int64')
    new_df = new_df[['MUNICIPIO', 'PRODUTO', valor]].groupby(['MUNICIPIO', 'PRODUTO']).mean().reset_index()
    new_df['PRODUTO'] = new_df['PRODUTO'].apply(limpar_nome)

    return new_df

area_colhida = pd.read_csv("../Dados/PAM/pam_area_colhida.csv",sep=';')
area_plantada = pd.read_csv("../Dados/PAM/pam_area_plantada.csv",sep=';')
rend_medio = pd.read_csv("../Dados/PAM/pam_rendimento_medio.csv",sep=';')
valor_producao = pd.read_csv("../Dados/PAM/pam_valor_producao.csv",sep=';')

area_colhida = limpar_dataframe(area_colhida, 'AREA_COLHIDA')
area_plantada = limpar_dataframe(area_plantada, 'AREA_PLANTADA')
rend_medio = limpar_dataframe(rend_medio, 'REND_MEDIO')
valor_producao = limpar_dataframe(valor_producao, 'VALOR_PROD')

agro = area_plantada.merge(area_colhida, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro = agro.merge(rend_medio, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro = agro.merge(valor_producao, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro['MUNICIPIO'] = agro['MUNICIPIO'].str.strip()
agro= agro[(agro['AREA_PLANTADA']>0)&(agro['AREA_COLHIDA']>0)&(agro['REND_MEDIO']>0)&(agro['VALOR_PROD']>0)]
agro.to_csv("../Dados/Tabela_final/dados_producao_agricola.csv")
agro

,MUNICIPIO,PRODUTO,AREA_PLANTADA,AREA_COLHIDA,REND_MEDIO,VALOR_PROD
2,ABAIARA,ALGODAO HERBACEO (EM CAROCO),26.666667,26.666667,2796.333333,214.666667
3,ABAIARA,AMENDOIM (EM CASCA),15.000000,15.000000,978.000000,60.666667
4,ABAIARA,BANANA,26.666667,26.666667,18363.333333,786.333333
12,ABAIARA,CASTANHA DE CAJU,7.000000,7.000000,333.666667,7.333333
14,ABAIARA,FAVA (EM GRAO),37.666667,37.666667,450.000000,86.666667
...,...,...,...,...,...,...
51870,XIQUE-XIQUE,TOMATE,1.000000,1.000000,9333.333333,24.666667
51891,ZABELE,FEIJAO (EM GRAO),193.333333,95.000000,219.000000,87.666667
51898,ZABELE,MANGA,4.000000,4.000000,6666.666667,31.333333
51901,ZABELE,MILHO (EM GRAO),193.333333,95.000000,429.666667,52.333333


## 2.7) Tabela final de dados socioeconômicos dos municípios

In [39]:
mun_ser = pd.read_csv('../Dados/Views/municipios_sertao.csv').drop(columns = 'Unnamed: 0')
reg_adm_mun = pd.read_csv('../Dados/Views/registros_administrativos.csv').drop(columns = 'Unnamed: 0')
met = pd.read_csv('../Dados/Views/dados_meteorologicos.csv').drop(columns = 'Unnamed: 0')
rec_h = pd.read_csv('../Dados/Views/recursos_hidricos.csv').drop(columns = 'Unnamed: 0')
censo = pd.read_csv('../Dados/Views/censo.csv').drop(columns = 'Unnamed: 0')
ql_agua = pd.read_csv('../Dados/Views/qualidade_da_agua.csv').drop(columns = 'Unnamed: 0')
trans = pd.read_csv('../Dados/Views/custo_de_transporte.csv').drop(columns = 'Unnamed: 0')
edu = pd.read_csv(r"..\Dados\Views\dados_edu_2020.csv")

tabela_final = mun_ser.copy()
tabela_final = tabela_final.merge(reg_adm_mun, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(met, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(rec_h, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(censo, right_on='Codmun7', left_on= 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(ql_agua, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(trans, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(edu, right_on='COD_MUNIC', left_on= 'IBGE7', how = 'left')

num_metrics = [column for column in tabela_final.columns if column not in ['IBGE7', 'UF', 'NOME', 
                                                                           'LATITUDE', 'LONGITUDE', 
                                                                           "Codmun7", "COD_MUNIC"]]
tabela_final.drop(columns=["Codmun7", "COD_MUNIC"], inplace=True)
tabela_final[num_metrics] = imputer.fit_transform(tabela_final[num_metrics])

#min_max_scaler = MinMaxScaler()
#tabela_final[num_metrics] = min_max_scaler.fit_transform(tabela_final[num_metrics])
#tabela_final[num_metrics] = knn_imputer.fit_transform(tabela_final[num_metrics])
#tabela_final[num_metrics] = min_max_scaler.inverse_transform(tabela_final[num_metrics])

tabela_final.to_csv('../Dados/Dados_Imputacao/tabela_final_knn.csv', index=False)